In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb
using DomainSets
import ApproxFun as AF
using NonlinearSolve
import DifferentialEquations as DE
include("multiharmonic_balance.jl");
using Random
using Base.Threads

Equation parameters

In [ ]:
gamma = 0.25;
gamma3 = 0.0;
omega=10.0
g0::Float64 = 3; # m / s^2
height = 3; # m
Nx = 100;
harmonics = 4; # number of harmonics

Discretization parameters

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
stepx = (xright-xleft)/Nx;

**Declare the problem**

In [ ]:
# Define symbolics
Model.@parameters x, t;

const Dx = Model.Differential(x);
const Dt = Model.Differential(t);

In [ ]:
u0 = rand((Nx+1) * harmonics * 2)*.4;
# u0[1] = 0.3;

sol = nothing 

u_hist = Vector{Float64}[]  # store history for prediction
gamma3_hist = Float64[]

gamma3 = 0.0
dgamma3 = 0.005

while gamma3 <= 0.0
    timed_out = false
    
    if length(u_hist) >= 2
        u0 = u_hist[end] + (u_hist[end] - u_hist[end-1]) * 
             dgamma3 / (gamma3_hist[end] - gamma3_hist[end-1])
    elseif length(u_hist) == 1
        u0 = u_hist[end]
    else
        u0 = rand((Nx+1) * harmonics * 2)*.3
    end
    
    omega = omega * 1.0;
    
    vars, var_exprs, (u,) = create_ansatz((x,), t, omega, harmonics);
    
    F = 100*exp(-40*x^2)* sin(omega*t);
    
    pde::Symbolics.Num = Dt(Dt(u)) - g0*height*(Dx(Dx(u))) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u);
    
    expanded = expand_trig_jl(pde, t, omega)
    eqns = make_residual(expanded, harmonics, omega, t)
    sym_eqs = map(transform_sym(Nx,) ∘ Meta.parse ∘ string, eqns)
    
    resid = create_residual_function(sym_eqs, vars, Nx, 0,
        Dict(
        :A1 => [:(0.3), nothing],
        :B1 => [:(0.0), nothing],
        :A2 => [:(0.0), nothing],
        :B2 => [:(0.0), nothing],
        :A3 => [:(0.0), nothing],
        :B3 => [:(0.0), nothing],
        :A4 => [:(0.0), nothing],
        :B4 => [:(0.0), nothing]));
    
    residual! = eval(resid)
    N = (Nx+1)
    
    f = NonlinearFunction(residual!)
    prob = NonlinearProblem(f, u0, [stepx])

    tsk = @task solve(prob, RobustMultiNewton(), reltol=1e-5, abstol=1e-5, maxiters=1000)
    schedule(tsk)
    
    Timer(20) do timer
        istaskdone(tsk) || Base.throwto(tsk, InterruptException())
    end

    try 
        sol = fetch(tsk)
        u0 = sol.u

        if sol.retcode == ReturnCode.Success
            push!(u_hist, copy(sol.u))
            push!(gamma3_hist, gamma3)
            dgamma3 = min(dgamma3 * 1.2, 0.02)
            gamma3 += dgamma3
        else
            println("Failed! Going back to the previous gamma and reducing the step!")
            gamma3 -= dgamma3
            dgamma3 /= 10
            gamma3 += dgamma3
            dgamma3 < 1e-6 && break
        end
    catch _;
        println("Timed out! Going back to the previous gamma and reducing the step!")
        gamma3 -= dgamma3
        dgamma3 /= 10
        gamma3 += dgamma3
        dgamma3 < 1e-6 && break
    end
    
end

In [ ]:
sol

In [ ]:
N = Nx + 1

solutions = [reshape(sol.u[(k-1)*N+1:k*N], Nx+1) for k in 1:(2*harmonics)];

In [ ]:
using Plots

xgrid = collect(range(start=0.0, stop=1.0, step=stepx))

anim = @animate for t in 1:0.01:10
    if t % 100 == 0
        println(t)
    end
    u_new = zeros(length(xgrid))
        j = 1
        for i in 1:(2*harmonics)
            if isodd(i)
                u_new .+= solutions[i] .* sin(j * omega * t)
            else
                u_new .+= solutions[i] .* cos(j * omega * t)
                j += 1
            end
        end
    plot(u_new)
    title!("Wave Equation with H = $(harmonics), \$\\gamma\$ = $(gamma), \$\\gamma_3\$ = $(gamma3), \$\\omega\$ = $omega", titlefontsize=10)
    ylims!(-1, 1)
end

gif(anim, "HB_WE_1D.gif", fps=60)